### 层和块

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [37]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [38]:
X = torch.rand(2, 20)
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0224, -0.0595, -0.0453, -0.0468, -0.1511, -0.0721,  0.0234,  0.1092,
         -0.1681, -0.2620],
        [ 0.0870, -0.0789,  0.0490,  0.0457, -0.1018,  0.0605,  0.0926,  0.0585,
         -0.1600, -0.3684]], grad_fn=<AddmmBackward0>)

In [39]:
net = MLP()
net(X)

tensor([[-0.1912, -0.2915, -0.0731,  0.0830, -0.0023,  0.0942,  0.0669, -0.0523,
         -0.1420, -0.1477],
        [-0.0753, -0.3063, -0.0558,  0.0429,  0.0058, -0.0848, -0.0199,  0.0221,
         -0.0729, -0.0871]], grad_fn=<AddmmBackward0>)

In [40]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.2464, -0.1436,  0.0585,  0.2172,  0.0201,  0.0946, -0.0070, -0.3157,
         -0.0600,  0.1383],
        [ 0.3651, -0.2100, -0.0468,  0.1550, -0.0066,  0.2223, -0.1252, -0.3472,
         -0.1809,  0.2063]], grad_fn=<AddmmBackward0>)

In [41]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)  #不参与训练
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


net = FixedHiddenMLP()
net(X)

tensor(-0.5427, grad_fn=<SumBackward0>)

In [42]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
print(X.shape)
chimera(X)

torch.Size([2, 20])


tensor(-0.1117, grad_fn=<SumBackward0>)

### 2. 参数管理

In [62]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3633],
        [-0.1087]], grad_fn=<AddmmBackward0>)

In [63]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3287, -0.2408, -0.3277,  0.1848,  0.2537, -0.2065,  0.0992, -0.2277]])), ('bias', tensor([-0.2038]))])


In [64]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data[0])
print(net[2].weight)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2038], requires_grad=True)
tensor(-0.2038)
Parameter containing:
tensor([[ 0.3287, -0.2408, -0.3277,  0.1848,  0.2537, -0.2065,  0.0992, -0.2277]],
       requires_grad=True)


In [65]:
net[2].weight.grad == None

True

In [47]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [48]:
net.state_dict()['2.bias'].data

tensor([0.1426])

In [49]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.0536],
        [-0.0536]], grad_fn=<AddmmBackward0>)

In [50]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


#### 内置初始化

In [66]:
def init_normal(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight, mean=0, std=0.01)
        torch.nn.init.zeros_(m.bias)


net.apply(init_normal)
print(net[0].weight.data, net[0].bias.data)
net[0].weight.data[0], net[0].bias.data[0]


tensor([[ 0.0095,  0.0085,  0.0132,  0.0126],
        [ 0.0056,  0.0070,  0.0054, -0.0056],
        [ 0.0113, -0.0164,  0.0167, -0.0089],
        [ 0.0065,  0.0137,  0.0052, -0.0061],
        [-0.0121, -0.0092, -0.0110,  0.0042],
        [ 0.0094,  0.0122, -0.0183,  0.0034],
        [ 0.0002, -0.0126,  0.0064, -0.0118],
        [-0.0018,  0.0114, -0.0049, -0.0107]]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])


(tensor([0.0095, 0.0085, 0.0132, 0.0126]), tensor(0.))

In [17]:
def init_constant(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, val=0.01)
        torch.nn.init.zeros_(m.bias)


net.apply(init_constant)
net[0].weight.data, net[0].bias.data

(tensor([[0.0100, 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [52]:
def xavier(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)


def init_42(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 42)


net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0844,  0.1932, -0.5143, -0.2162])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### 自定义初始化

In [53]:
def my_init(m):
    if type(m) == nn.Linear:
        print('Init', *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5


net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[0.0000, 0.0000, 0.0000, -0.0000],
        [0.0000, 9.0615, 0.0000, -0.0000]], grad_fn=<SliceBackward0>)

In [55]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  2.,  2.,  2.])

#### 参数绑定

In [56]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 自定义层

In [57]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()


layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [58]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-6.5193e-09, grad_fn=<MeanBackward0>)

In [59]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)


dense = MyLinear(5, 3)
print(dense.weight)
dense(torch.rand(2, 5))

Parameter containing:
tensor([[ 0.8103,  0.7537, -1.1715],
        [ 0.6664, -2.1664,  0.9121],
        [ 1.2790,  1.6284, -0.7778],
        [-1.8282,  0.4328, -1.1373],
        [-1.7114, -0.4641, -0.6634]], requires_grad=True)


tensor([[0.3583, 0.0000, 0.0000],
        [0.4513, 2.1888, 0.0000]])

#### 自定义层直接执行正向传播

In [60]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[7.0978],
        [3.5106]])

### 读写文件

In [26]:
x = torch.arange(4)
torch.save(x, './data/x-file')
x2 = torch.load('./data/x-file')
x2

tensor([0, 1, 2, 3])

In [27]:
y = torch.zeros(4)
torch.save([x, y], './data/x-files')
x2, y2 = torch.load('./data/x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [28]:
mydict = {'x': x, 'y': y}
torch.save(mydict, './data/mydict')
mydict2 = torch.load('./data/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

#### 加载保存模型

In [29]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    def forward(self,x):
        return self.out(F.relu(self.hidden(x)))
net=MLP()
x=torch.randn(size=(2,20))
y=net(x)

In [30]:
torch.save(net.state_dict(),'./data/mlp.params')

In [31]:
clone=MLP()
clone.load_state_dict(torch.load('./data/mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)

In [32]:
y_clone=clone(x)
y_clone==y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])